In [1]:
using Pkg
Pkg.activate(".")

In [2]:
using JuMP
using DataFrames
using CSV
using Gurobi
using DataFrames

In [3]:
include("./parameters.jl")
include("./get_functions.jl")
include("./modification_functions.jl")

construct_ef_model (generic function with 1 method)

In [4]:
function second_func(scen)
    
    model = JuMP.read_from_file("./storage_expansion_revised/second_stage/scen_$(scen).mps")
    JuMP.set_optimizer(model, Gurobi.Optimizer)
    set_optimizer_attribute(model, "OutputFlag", 0) 
    return model
end

second_func (generic function with 1 method)

In [5]:
function first_func()
    
    model = JuMP.read_from_file("./storage_expansion_revised/first_stage/first_stage_model.mps")
    JuMP.set_optimizer(model, Gurobi.Optimizer)
    set_optimizer_attribute(model, "OutputFlag", 0) 
    
    return model
end

first_func (generic function with 1 method)

In [6]:
model1 = second_func(1)

Academic license - for non-commercial use only - expires 2022-08-19


A JuMP Model
Minimization problem with:
Variables: 4440
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.EqualTo{Float64}`: 2064 constraints
`AffExpr`-in-`MathOptInterface.GreaterThan{Float64}`: 3456 constraints
`AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 2304 constraints
`AffExpr`-in-`MathOptInterface.Interval{Float64}`: 1488 constraints
`VariableRef`-in-`MathOptInterface.EqualTo{Float64}`: 48 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: Gurobi

In [7]:
model = JuMP.read_from_file("./ef_12scen_test_revised.mps")

A JuMP Model
Minimization problem with:
Variables: 52752
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.EqualTo{Float64}`: 24768 constraints
`AffExpr`-in-`MathOptInterface.GreaterThan{Float64}`: 41520 constraints
`AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 27673 constraints
`AffExpr`-in-`MathOptInterface.Interval{Float64}`: 17856 constraints
`VariableRef`-in-`MathOptInterface.EqualTo{Float64}`: 576 constraints
Model mode: AUTOMATIC
CachingOptimizer state: NO_OPTIMIZER
Solver name: No optimizer attached.

In [8]:
for scen = 1:12
    println(scen,"...")
    for ts in timesteps
        print(ts,".")
        for gen in gens
            con = get_thermal_interval(model,gen,ts,scen)
            conname = JuMP.name(con)
            var = get_thermal_variable(model,gen,ts,scen)
            lb = JuMP.constraint_object(con).set.lower
            ub = JuMP.constraint_object(con).set.upper

            JuMP.delete(model,con)
            JuMP.unregister(model, Symbol(conname))

            JuMP.@constraint(model, var >= lb, base_name = string("Tlb_{",string(gen),",",string(ts),"}_",string(scen)))
            JuMP.@constraint(model, var <= ub, base_name = string("Tub_{",string(gen),",",string(ts),"}_",string(scen)))
        end
        
        for branch in branches
            con = get_branch_interval(model,branch,ts,scen)
            conname = JuMP.name(con)
            var = get_line_variable(model,branch,ts,scen)
            lb = JuMP.constraint_object(con).set.lower
            ub = JuMP.constraint_object(con).set.upper
            coeff = 0
            for (key, value) in JuMP.constraint_object(con).func.terms
                coeff = value
            end

            JuMP.delete(model,con)
            JuMP.unregister(model, Symbol(conname))

            JuMP.@constraint(model,var >= lb/coeff, base_name=string("Blb_{",string(branch),",",string(ts),"}_",string(scen)))
            JuMP.@constraint(model,var <= ub/coeff, base_name=string("Bub_{",string(branch),",",string(ts),"}_",string(scen)))
        end
    end
end
            

1...
1.2.3.4.5.6.7.8.9.10.11.12.13.14.15.16.17.18.19.20.21.22.23.24.2...
1.2.3.4.5.6.7.8.9.10.11.12.13.14.15.16.17.18.19.20.21.22.23.24.3...
1.2.3.4.5.6.7.8.9.10.11.12.13.14.15.16.17.18.19.20.21.22.23.24.4...
1.2.3.4.5.6.7.8.9.10.11.12.13.14.15.16.17.18.19.20.21.22.23.24.5...
1.2.3.4.5.6.7.8.9.10.11.12.13.14.15.16.17.18.19.20.21.22.23.24.6...
1.2.3.4.5.6.7.8.9.10.11.12.13.14.15.16.17.18.19.20.21.22.23.24.7...
1.2.3.4.5.6.7.8.9.10.11.12.13.14.15.16.17.18.19.20.21.22.23.24.8...
1.2.3.4.5.6.7.8.9.10.11.12.13.14.15.16.17.18.19.20.21.22.23.24.9...
1.2.3.4.5.6.7.8.9.10.11.12.13.14.15.16.17.18.19.20.21.22.23.24.10...
1.2.3.4.5.6.7.8.9.10.11.12.13.14.15.16.17.18.19.20.21.22.23.24.11...
1.2.3.4.5.6.7.8.9.10.11.12.13.14.15.16.17.18.19.20.21.22.23.24.12...
1.2.3.4.5.6.7.8.9.10.11.12.13.14.15.16.17.18.19.20.21.22.23.24.

In [12]:
JuMP.set_optimizer(model, Gurobi.Optimizer)
JuMP.set_optimizer_attribute(model, "Method", 1)  

JuMP.optimize!(model)

Academic license - for non-commercial use only - expires 2022-08-19
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 129673 rows, 52752 columns and 1439880 nonzeros
Model fingerprint: 0x5a2f6d24
Coefficient statistics:
  Matrix range     [2e-04, 6e+01]
  Objective range  [5e-03, 6e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e-03, 2e+02]
Presolve removed 86772 rows and 3800 columns
Presolve time: 0.52s
Presolved: 42901 rows, 49780 columns, 1007088 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0      handle free variables                          1s
    5941    3.2263239e+02   5.212213e+06   0.000000e+00      5s
    9103    5.4081643e+02   1.132073e+07   0.000000e+00     10s
   12696    5.9061290e+02   2.221129e+07   0.000000e+00     15s
   15959    6.1972526

In [13]:
for bus in buses
    varPR = get_PR_variable(model,bus,0)
    varER = get_ER_variable(model,bus,0)
    
    println(varPR, " ", JuMP.value(varPR))
    println(varER, " ", JuMP.value(varER))
end

PR_{101}_0 0.0
ER_{101}_0 0.0
PR_{102}_0 0.0
ER_{102}_0 0.0
PR_{103}_0 0.0
ER_{103}_0 0.0
PR_{104}_0 0.0
ER_{104}_0 0.0
PR_{105}_0 0.0
ER_{105}_0 0.0
PR_{106}_0 0.0
ER_{106}_0 0.0
PR_{107}_0 0.0
ER_{107}_0 0.0
PR_{108}_0 0.0
ER_{108}_0 0.0
PR_{109}_0 0.0
ER_{109}_0 0.0
PR_{110}_0 0.0
ER_{110}_0 0.0
PR_{111}_0 0.0
ER_{111}_0 0.0
PR_{112}_0 0.0
ER_{112}_0 0.0
PR_{113}_0 0.0
ER_{113}_0 0.0
PR_{114}_0 0.2970924254297301
ER_{114}_0 1.4854621271486503
PR_{115}_0 0.0
ER_{115}_0 0.0
PR_{116}_0 1.6240825552933416
ER_{116}_0 8.120412776466708
PR_{117}_0 1.4697830484937653
ER_{117}_0 7.348915242468826
PR_{118}_0 0.0
ER_{118}_0 0.0
PR_{119}_0 0.0
ER_{119}_0 0.0
PR_{120}_0 0.0
ER_{120}_0 0.0
PR_{121}_0 0.0
ER_{121}_0 0.0
PR_{122}_0 0.08005897876650214
ER_{122}_0 0.4002948938325107
PR_{123}_0 0.0
ER_{123}_0 0.0
PR_{124}_0 0.0
ER_{124}_0 0.0


In [14]:
model

A JuMP Model
Minimization problem with:
Variables: 52752
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.EqualTo{Float64}`: 24768 constraints
`AffExpr`-in-`MathOptInterface.GreaterThan{Float64}`: 59376 constraints
`AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 45529 constraints
`VariableRef`-in-`MathOptInterface.EqualTo{Float64}`: 576 constraints
Model mode: AUTOMATIC
CachingOptimizer state: ATTACHED_OPTIMIZER
Solver name: Gurobi

In [15]:
JuMP.write_to_file(model, "./ef_12scen_test_revised_noint.mps")

In [17]:
for scen = 1:12
    println(scen,"...")
    model = JuMP.read_from_file("./storage_expansion_revised/second_stage/scen_$(scen).mps")
    for ts in timesteps
        print(ts,".")
        for gen in gens
            con = get_thermal_interval(model,gen,ts)
            conname = JuMP.name(con)
            var = get_thermal_variable(model,gen,ts)
            lb = JuMP.constraint_object(con).set.lower
            ub = JuMP.constraint_object(con).set.upper

            JuMP.delete(model,con)
            JuMP.unregister(model, Symbol(conname))

            JuMP.@constraint(model, var >= lb, base_name = string("Tlb_{",string(gen),",",string(ts),"}"))
            JuMP.@constraint(model, var <= ub, base_name = string("Tub_{",string(gen),",",string(ts),"}"))
        end
        
        for branch in branches
            con = get_branch_interval(model,branch,ts)
            conname = JuMP.name(con)
            var = get_line_variable(model,branch,ts)
            lb = JuMP.constraint_object(con).set.lower
            ub = JuMP.constraint_object(con).set.upper
            coeff = 0
            for (key, value) in JuMP.constraint_object(con).func.terms
                coeff = value
            end

            JuMP.delete(model,con)
            JuMP.unregister(model, Symbol(conname))

            JuMP.@constraint(model,var >= lb/coeff, base_name=string("Blb_{",string(branch),",",string(ts),"}"))
            JuMP.@constraint(model,var <= ub/coeff, base_name=string("Bub_{",string(branch),",",string(ts),"}"))
        end
    end
    println("writing...")
    JuMP.write_to_file(model,"./storage_expansion_revised/second_stage/noint_scen_$(scen).mps")
end
            

1...
1.2.3.4.5.6.7.8.9.10.11.12.13.14.15.16.17.18.19.20.21.22.23.24.writing...
2...
1.2.3.4.5.6.7.8.9.10.11.12.13.14.15.16.17.18.19.20.21.22.23.24.writing...
3...
1.2.3.4.5.6.7.8.9.10.11.12.13.14.15.16.17.18.19.20.21.22.23.24.writing...
4...
1.2.3.4.5.6.7.8.9.10.11.12.13.14.15.16.17.18.19.20.21.22.23.24.writing...
5...
1.2.3.4.5.6.7.8.9.10.11.12.13.14.15.16.17.18.19.20.21.22.23.24.writing...
6...
1.2.3.4.5.6.7.8.9.10.11.12.13.14.15.16.17.18.19.20.21.22.23.24.writing...
7...
1.2.3.4.5.6.7.8.9.10.11.12.13.14.15.16.17.18.19.20.21.22.23.24.writing...
8...
1.2.3.4.5.6.7.8.9.10.11.12.13.14.15.16.17.18.19.20.21.22.23.24.writing...
9...
1.2.3.4.5.6.7.8.9.10.11.12.13.14.15.16.17.18.19.20.21.22.23.24.writing...
10...
1.2.3.4.5.6.7.8.9.10.11.12.13.14.15.16.17.18.19.20.21.22.23.24.writing...
11...
1.2.3.4.5.6.7.8.9.10.11.12.13.14.15.16.17.18.19.20.21.22.23.24.writing...
12...
1.2.3.4.5.6.7.8.9.10.11.12.13.14.15.16.17.18.19.20.21.22.23.24.writing...


# determine scenarios needed based on Yan et al.

In [1]:
function scen_conf(K, eps, S)
    
    sum = 0
    
    for i = 0:2*S
        #println("i=$(i)")
        sum += binomial(BigInt(K),BigInt(i))/BigFloat(eps^(-i))/BigFloat((1-eps)^-(K-i))
    end
    
    return sum
    
end

scen_conf (generic function with 1 method)

In [2]:
for K = 10:1000
    sum = scen_conf(K,.5,24)
    println(K, " ", sum)
end

10 1.0
11 1.0
12 1.0
13 1.0
14 1.0
15 1.0
16 1.0
17 1.0
18 1.0
19 1.0
20 1.0
21 1.0
22 1.0
23 1.0
24 1.0
25 1.0
26 1.0
27 1.0
28 1.0
29 1.0
30 1.0
31 1.0
32 1.0
33 1.0
34 1.0
35 1.0
36 1.0
37 1.0
38 1.0
39 1.0
40 1.0
41 1.0
42 1.0
43 1.0
44 1.0
45 1.0
46 1.0
47 1.0
48 1.0
49 0.9999999999999982236431605997495353221893310546875
50 0.99999999999995470290059529361315071582794189453125
51 0.999999999999410693618528966908343136310577392578125
52 0.9999999999947866147209651899174787104129791259765625
53 0.99999999996473010188680063947685994207859039306640625
54 0.999999999805430583865728522141580469906330108642578125
55 0.9999999990885827527709039941328228451311588287353515625
56 0.99999999627239484489837906266984646208584308624267578125
57 0.999999986415737167344541802549429121427237987518310546875
58 0.9999999552029878550907238121681075426749885082244873046875
59 0.99999986468601484955465164006227496429346501827239990234375
60 0.999999621935950880162458087596633049543015658855438232421875
6

158 4.462132224920480408759407559926630459764829109246602089764789076266184102767176e-07
159 3.175828906149100120298306544933734981896974769241865539320905408325557613778574e-07
160 2.254557610272300724508599061222607139640268282481716388327313051557271074367819e-07
161 1.596506684646015441801665144286087252314049363367324137617604225294209260502993e-07
162 1.127718193381272386421946734433079721962185443113265941315555017204151950625839e-07
163 7.946316337984286365468836537480480556595452892485403807851516325085849146604926e-08
164 5.585746372245002398962954704799603964972394410748435704962135816156395804763558e-08
165 3.917067603360336146777762374801053270894025484003337563264470283982127063117073e-08
166 2.740435138121148404852306244673870089172098676683076053093039460013091411956091e-08
167 1.912803827825787535531858288736953105418539990178146346786016083831481589529298e-08
168 1.332070933626941883529695227218192028583059735361662057066382034157830999843271e-08
169 9.255579076877499

263 1.035507479187591814459857974469585438623807745574845625053941652603501617938283e-26
264 6.319939333056631515526455186546312555008812002204504871011009589204130588487559e-27
265 3.854023219333734131093490178788259213702038669483201250188094238918589394051563e-27
266 2.348337113028278354884421683267788855530852187522036596920645925956219770720827e-27
267 1.429730451841922307931182647009887214995220067793436143321698101992939220707122e-27
268 8.697578981050340327336602207978786806961018578218920311963120997139531320001371e-28
269 5.286837062834748105678965611960189370775480390210424356290315346894797870604282e-28
270 3.21106879134154831467013247999411989038201936628670170453469380862368181203456e-28
271 1.948777274892980874191788007852591152304914689576329821710329359675030154796216e-28
272 1.181779514898447743318175111102917860782727767180565740442610154147934641093096e-28
273 7.16102303473195485287767280933473362358542850011708976815780636506483793487631e-29
274 4.33591461875209115

369 5.303201653479393861746475684576643639048822811552846070888891225242551759207547e-51
370 3.045235662202904874742284381313716686592087680100404685242775333179148692683135e-51
371 1.747957064730387910159130992793090953037752278489529976098267755440858110984326e-51
372 1.00292554822527367817716519226499165404602095651176756573818677813295438681984e-51
373 5.7522227023159698944677741788774946388410112352453358942036251338212076739208e-52
374 3.297863891675332896061087412066550993757994347487823999333648968404885519358421e-52
375 1.889995494160611673048662794294629332314668709418330979883693315236627105545032e-52
376 1.082731504759968769945208311627243058551293916947291404511012321447488106798823e-52
377 6.200314132742344230444478154642289748088717797992565259437439469585913636150196e-53
378 3.549281693682741665374771360516813918743837164180876061567224010888131384170049e-53
379 2.030963114948605650834848378426768671028132801179635884605191520906674276217783e-53
380 1.161714260960331406

465 8.440190136197292643622067757118613549149571073003542328202613596523538764247729e-75
466 4.70325326798752065719700866902668512432899526114568365546420557408965410757605e-75
467 2.620223865372982683902657550544820715278291519129221763722461389335838019048535e-75
468 1.45939483074173061525294159310922324627270840752816245173750371119737487663523e-75
469 8.126472257328901770052427025379617992553121024932865493458844333773739830049598e-76
470 4.524042010901987690026645794050382616648621416128152925505525553517440315648213e-76
471 2.517949338128706152571530653381258651393746041556808249038630024227794851467348e-76
472 1.401082140875496218598292791377526514709294148621697914977273293382141171054596e-76
473 7.794296442911623874245094530924303254226652648181931063959520941378622357305173e-77
474 4.334983138389389731360394307290767989137529565369486658559933562054576046619418e-77
475 2.410435595732653834966793478649575834616116019297774630203825090458803803350582e-77
476 1.33999053395051150

566 6.449781573799781744640284096277117599801869344698051157323696536911100966533666e-101
567 3.52245986006391814770457992986667345729491712231599393455002503205641855660066e-101
568 1.923431525104154506603862914110679402572911428587066723266140886340999552330896e-101
569 1.050116049856898979541163620890098034224739088165883400180327237527193788460487e-101
570 5.732288085750483126518393427302988033321190673408802227946957920616894893796204e-102
571 3.128593377602447876260588460338566945114357226375738903140349453994888663182277e-102
572 1.707264813211885612496528578754738607288255985213330529847258216479434019128213e-102
573 9.315015128002428502130913151421910793908037810674358833552275028736706065338184e-103
574 5.081563974327463713669869798564865713096227210906762334124335991630968585180204e-103
575 2.771677128120781405136943102234120583451760210653339909531525148191146043725185e-103
576 1.511539617144650164713382523116304217659000699129888397063910124492950634487389e-103
577 8.24191

660 6.902716572169071638923521548497449074653940871452860848751009220593818914636315e-126
661 3.721098776624916939195764067849541910689284118907443952489615440651167597355092e-126
662 2.005724092404128801740586502834642534359236718065257518208390898610309097581121e-126
663 1.08098627403754757454569436273703391291746849383274333295255160965994401056942e-126
664 5.825300353570245228138134774649082901403202558927783696805016514696252685460397e-127
665 3.138815430811582027245530003377237012409481536264336167481630376397781543386335e-127
666 1.691074366277942619957375892440626383413675636692867280123403648708573026790507e-127
667 9.109808800486514830205549880039090056537996422635806466439708002935140688189684e-128
668 4.906881859331933341216377478913964927573236420339084458921277389907319000249328e-128
669 2.642724442645432990696403959598171712937381838456656796806509930631040865133072e-128
670 1.423142066024443671454968947986041551430629249664914360450004753291403125686393e-128
671 7.66293

755 1.372243798824808840918589152103081842761328345015372159919435997815154556472141e-151
756 7.325637602018124851708276414224142416000706023722487444721042485122950810242649e-152
757 3.910396717844500240512600552115493358953312990966828531739525518505989204037887e-152
758 2.087168685884391756256516893711933883505586244989371693145020339571546371952552e-152
759 1.113924426570474692125452518310597205766588446784503887754432363774146606656296e-152
760 5.944501700168438540976903094993352237666592422574330971134645201523277445888847e-153
761 3.172026735415914405435475576056841659577082623132099223331726935002334080922327e-153
762 1.692466112571124176019608576119475215028613508562997989055415012966963884050293e-153
763 9.029526709690722468775282862369309357947665440660406077399040293598545773160557e-154
764 4.816947996807046790905302014954055616301335133589227881988306863582990521145013e-154
765 2.569454884626091331483049663400107391009577902721643984436770312932233389483089e-154
766 1.3704

850 1.207961235367458079425975125479030073761638385056475260515837890982019754706121e-177
851 6.400336102300394468592779102543006347130410830605892352050260218108982856494733e-178
852 3.390956220656912175228773310785567040590209684911627049811773912600736466910552e-178
853 1.796434044562231258595606062914088005035326988312426777730187885055322335115991e-178
854 9.516344071654841642551764589175838948935164664496206708447264679769134814384862e-179
855 5.040792642766367581964128845918925114809195770756526712912624169936869547383641e-179
856 2.669915027091246564738336677093236559556591431136064261746504197256208578804962e-179
857 1.414054111857395332841506171824183710982192102723245042564450548361006976637045e-179
858 7.488669645425557124857683825685729005815368342128704623178374599165151515085507e-180
859 3.965641420758073209640257756665271750730416364462646661131497871477657774590188e-180
860 2.099869999679547511014013001246916151397510230791164670133799283879903691306903e-180
861 1.1118

946 2.961762111938198505414380931329025718320743413024688478611095689627827541358117e-204
947 1.559851734531734917767094432831832576852808486821760237749452756446936378203496e-204
948 8.214706848076409147070386407979939200284845852009743400320023350218507655742486e-205
949 4.32589998619618073095409256993505560767673997014027100567478446404639009589496e-205
950 2.277910134961809606151449477646157422668364563983706464876598169386379238905371e-205
951 1.199423239910450089209924567627059154509852016395715160132542193817083055900902e-205
952 6.315156888618106093586564671684974418748877679149954863719678678113241023918816e-206
953 3.324847217167305292599799009978388409741765043344395519582141297817430604999691e-206
954 1.75039135148077868301947432732293179836951181576279438424342686498764755017655e-206
955 9.214559784868789117503629877791649069516698846850639851743686039944836239616967e-207
956 4.850539491543682824438572935871597353397409738916612668111102582125257686501398e-207
957 2.553180

In [3]:
function scen_conf_cautious(K, eps, S)
    sum = 0
    
    for i = 0:2*S
        summand = binomial(BigInt(K),BigInt(i))
        if i > 0
            for j = 1:i
                summand = BigFloat(summand)*BigFloat(eps)
            end
        end
        if K-i > 0
            for j = 1:K-i
                summand = BigFloat(summand)BigFloat(1-eps)
            end
        end
        sum += summand
    end
    return sum
end

scen_conf_cautious (generic function with 1 method)

In [7]:
for i = 723:1500
    val1 = scen_conf(i,.05,24)
    val2 = scen_conf(i,.05,24)
    println(i, " ", val1, " ", val1-val2)
end

723 0.9789142679481288223613987284894700806654445864860165213747624823578682237392532 0.0
724 0.9784521255127924163120935119936439358951875533450528570087638836885023664307045 0.0
725 0.9779819160941172981270917431452658679239845791931176335001140959648651448563918 0.0
726 0.9775035457291062400213355887334406801238469634645285844047903584147156018906827 0.0
727 0.977016920300274195277317634910225752954765062041518838986910097700800151657387 0.0
728 0.9765219455544378896392428788339431660309127394643521812574769839718949926213743 0.0
729 0.9760185271217608481360864946042898791437845561081953889170116559071970444802366 0.0
730 0.9755065705350494172223157852719678470205311017272613173550275899210540887835446 0.0
731 0.9749859812492951985266145490676817168119185423428220642400204435779004970315086 0.0
732 0.9744566646614591561924400777219351743646494679366842519301579909546915899440426 0.0
733 0.9739185261304925141429532990092329509242105016879261055013954773138300023613608 0.0
734 0.97337

818 0.8867399511326037184994494807652084669332285625015886510169372673306184090569676 0.0
819 0.8851398961116594138131103322275304536735712861672163432488974408716889822639375 0.0
820 0.8835252102374107518248452968898737260081710092175974809166432451243550191708094 0.0
821 0.881895883429069165910283992749906047087632087093153533312961341688410478237278 0.0
822 0.8802519074340290382307713079833367471148426762665913633517709805312154630364598 0.0
823 0.8785932758390331240254273397545261646028963628122774706695790049241614136867678 0.0
824 0.8769199840809072487003782839234844502687604933055367572837109592677775888555025 0.0
825 0.8752320294568596538050671583712960376333912067246699351569609449091021327482638 0.0
826 0.8735294111343406022190208730228846447799715372896688657717721121260510036106554 0.0
827 0.87181213016045795455263147103390296849395531856795618797264093495918890389838 0.0
828 0.8700801894709446038902342989228999914751443915264356329494766512335362488479151 0.0
829 0.8683335

912 0.6768415590782113481504343054182426896704443911858733278856813455142119139744576 0.0
913 0.6740756451606287942570113661560600850067503959691042566112459361846043424977567 0.0
914 0.6713022169104377824596081307704839122998031594133075976089977474768019030577223 0.0
915 0.6685214227422612730934029926555527106567487491156797042590637609604736088612212 0.0
916 0.6657334119802227094290175559787199784060383236251547369401930464682041080548087 0.0
917 0.6629383348314600901098078024897080233055667096517697564954246827314910629472471 0.0
918 0.6601363423594870629776200722623095063570506774891660980020487914953362319935322 0.0
919 0.6573275864574092745238764482244976314912344757589491619856857021791240055970538 0.0
920 0.6545122198210039767294384197145116035067038271337871744739999382848960584291533 0.0
921 0.6516903959216711468498008580434830789868082947114851532769970713437650408809977 0.0
922 0.6488622689792642063300818583137110917810649695953422067095111513221640969262964 0.0
923 0.6460

1004 0.4107600171462885068703115084317089403235447601714975294067716788222174579164381 0.0
1005 0.4079623780158482588894777506841685713150675757695840377304345128345075971411774 0.0
1006 0.4051713164068745682382797452123928458683908557374014254209244070696101429877734 0.0
1007 0.4023869556765027952409231646510360049735615656410064162274341255680148398913085 0.0
1008 0.3996094179364504965046368518993587593452418146497915149014998521237687038759406 0.0
1009 0.396838824040748329245471376361883603496768579297732888833515734028138223490467 0.0
1010 0.3940752935738125146619751689013656151782621765878501937310447784233543786810896 0.0
1011 0.3913189448388593841198402440481414029369358637461074285101744009829715635838982 0.0
1012 0.3885698948466623646546977644288526170399081161503524458448780087899680847784057 0.0
1013 0.3858282593046517657182786481844769674891331355551212464510819123324461762042195 0.0
1014 0.3830941526063575728695865956022049564114926751047975077669501108411513911322798 0.0


1097 0.190706969589217292008661146735338282768904548290880806895310666135369364262277 0.0
1098 0.188868545781569567850625008157511279150624813292285159847542992792643896925558 0.0
1099 0.1870422030446578162492738058975845414787166844804476066064572074590524184809723 0.0
1100 0.1852279374552451473634786489103729076401968453865634279386432575893369385059151 0.0
1101 0.1834257439752677340945614811588246301854224826199954649547534313712371835645546 0.0
1102 0.1816356164630337589948589469644983853594684100381119292336037847515854295919436 0.0
1103 0.1798575476845159712696527901541502229135273925828462022765083641676235823197297 0.0
1104 0.1780915293247341141930088860851254614427752794960032352217116512933613402045761 0.0
1105 0.1763375519992234924196193002581699278777058885972702660246781343294972267409612 0.0
1106 0.1745956052655860534630031575134319160469554164564600636438093075759720871988187 0.0
1107 0.1728656776351202171414778531514265316371576859286601373342991875616222167099169 0.0
1

1192 0.06659889346787408868426733204591148590770462022036650019606084014225817978381463 0.0
1193 0.0657744914464176864958765340494031308252109561318585251294919190054617302185876 0.0
1194 0.06495847744552894082935250008051252563854491473492970631466884100936641289712228 0.0
1195 0.06415079447763356296819122910603710311235330627872663837944735670125900009217072 0.0
1196 0.06335138550701174220070210874122971969286745864170725914223485972749161693989476 0.0
1197 0.06256019345786495414125960543497956210340759541409465191395623636430331609079177 0.0
1198 0.06177716122227986316612358027358118818706235811636526805865449063903142043392219 0.0
1199 0.061002231668089086276006973600171031688437783427888835715417169510817214117026 0.0
1200 0.06023534764662861339708940862020937674869456583488401473669758015285552172309494 0.0
1201 0.0594764520003916888402439406174759264401663851791770609646608375009342171024598 0.0
1202 0.05872548757057900765214180992913124193631707147972668345975282601427396780433

1284 0.01889881017223200329431876457662141673530448522503795369081235983134386541158767 0.0
1285 0.01861904397839619643398243944475178557175143314749286915980523441391418719274071 0.0
1286 0.01834295296697739189212143740562325048178452252120706178307576170252169854746828 0.0
1287 0.01807049707953768801870012367266026264238489126072768580297191124986370876630714 0.0
1288 0.01780163655429301164990678855660579406078013777478959217758532512316576958414373 0.0
1289 0.01753633192631770049385063345825051913533904203643026740436009590945499344066884 0.0
1290 0.01727454402769790415205865800221990039556493480747969531120449399831043980174006 0.0
1291 0.0170162339876346507385410379633227294350660283804498524818851359837462116815371 0.0
1292 0.01676136323249742078581222726458534289531897703102266294181479385932910366561934 0.0
1293 0.01650989348582906380066197124900588641442304742485091186935450170771089553705884 0.0
1294 0.01626178676830289852473181830272304918705674620675449659304652106851547411

1374 0.004474003444503725036316688346466814413352286680369404331628478610368181750228329 0.0
1375 0.004398186652601978434735518070050618794376220396053646416430507775674475339641134 0.0
1376 0.004323555390792139267142623493787954391412867340055848915233659490572185920113021 0.0
1377 0.004250093052360044804867831288235544839588125750675239834816522169434988645145511 0.0
1378 0.004177783226912381496898924439272874825188771757954012918571240436285540162139813 0.0
1379 0.004106609698721752815149541986347100298028615675242657709768974746329195842226876 0.0
1380 0.004036556445073268350365393601656086410535089625751139064147440887666028369027773 0.0
1381 0.003967607634612935512921986249187826838777004859893169168216771481257560347625448 0.0
1382 0.003899747625698131524290881934906890081585260925053832903300116385949426421350101 0.0
1383 0.003832960964750420961419879334248554249403559407097273085281834661771325412308135 0.0
1384 0.003767232384610983049662601542638885866142873068157179189451713

1465 0.0008625130722332831207581055154778822040797283787536035691872329797822615154630159 0.0
1466 0.0008462159693189057993163382891093297784418976492791916523458124003993415698233861 0.0
1467 0.0008302096398204197098755544011575874143519706394279177806004377972217986208732549 0.0
1468 0.0008144892584916211023278367370319436583274703641249977918470045708991153097224795 0.0
1469 0.00079905007271616866770501360261779715917624265492878071667720802583769234457093 0.0
1470 0.0007838874015628029563056401826909377369539106228932639832906425779747938245566959 0.0
1471 0.0007689966348499511954001323324258862994015424673574222518656198557925381750984012 0.0
1472 0.0007543732322196808621485007461515367855193075348739223649388053271141315145110766 0.0
1473 0.000740012722220965958324397358512523284867236738843025296631023341771125719765617 0.0
1474 0.0007259107014022278851615208763112545280203727551009283778517437752348857934572075 0.0
1475 0.00071206283341311223336485704620111676218015115311339778

In [28]:
scen_conf_cautious(80,.5,6)

6.008243175410135281148838056440586452566776642925105988979339599609375e-11

In [15]:
function scen_conf_exp(K, eps, S)
    
    sum = 0
    
    for i = 0:2*S
        #println("i=$(i)")
        sum += binomial(BigInt(K),BigInt(i))
        println(i, " ", sum)
    end
    
    return sum
    
end

scen_conf_exp (generic function with 1 method)

In [23]:
testans = scen_conf_exp(46,.5,24)

0 1
1 47
2 1082
3 16262
4 179447
5 1550201
6 10917020
7 64441700
8 325374515
9 1427090845
10 5503441266
11 18844224462
12 57754842117


57754842117

In [24]:
for i = 1:46
    testans = BigFloat(testans/2)
    println(i, " ", testans)
end

1 2.88774210585e+10
2 1.443871052925e+10
3 7.219355264625e+09
4 3.6096776323125e+09
5 1.80483881615625e+09
6 9.02419408078125e+08
7 4.512097040390625e+08
8 2.2560485201953125e+08
9 1.12802426009765625e+08
10 5.64012130048828125e+07
11 2.820060650244140625e+07
12 1.4100303251220703125e+07
13 7.0501516256103515625e+06
14 3.52507581280517578125e+06
15 1.762537906402587890625e+06
16 881268.9532012939453125
17 440634.47660064697265625
18 220317.238300323486328125
19 110158.6191501617431640625
20 55079.30957508087158203125
21 27539.654787540435791015625
22 13769.8273937702178955078125
23 6884.91369688510894775390625
24 3442.456848442554473876953125
25 1721.2284242212772369384765625
26 860.61421211063861846923828125
27 430.307106055319309234619140625
28 215.1535530276596546173095703125
29 107.57677651382982730865478515625
30 53.788388256914913654327392578125
31 26.8941941284574568271636962890625
32 13.44709706422872841358184814453125
33 6.723548532114364206790924072265625
34 3.361774266057182

In [22]:
6.008243175410135281148838056440586452566776642925105988979339599609375e-11 - testans

0.0